In [ ]:
%load_ext autoreload
%autoreload 2

import os
import shutil
import pickle
import time
import pprint
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
from IPython.display import SVG

from pydrake.examples import QuadrotorGeometry
from pydrake.geometry import MeshcatVisualizerCpp, Rgba, StartMeshcat
from pydrake.geometry.optimization import HPolyhedron, VPolytope
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.multibody.parsing import Parser
from pydrake.perception import PointCloud
from pydrake.solvers import GurobiSolver, MosekSolver
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder

from reproduction.uav.helpers import *
from reproduction.util import *

g_lic = GurobiSolver.AcquireLicense()
m_lic = MosekSolver.AcquireLicense()

In [ ]:
runs = 100
save_location = os.path.join(GcsDir(), "data/uav/statistics")

# Generate Buildings to Plan Through

In [ ]:
np.random.seed(0)
generate_buildings(save_location, runs)

# Plan Through Buildings
## Using GCS and Mosek

In [ ]:
start_time = time.time()
plan_through_buildings(save_location, runs, solve_gcs=True, solver=MosekSolver())
print("Solved", runs, "buildings in", np.round((time.time()-start_time)/60., 4), "minutes.")

## Using MICP and Mosek

In [ ]:
start_time = time.time()
plan_through_buildings(save_location, runs, solve_gcs=False, solver=MosekSolver())
print("Solved", runs, "buildings in", np.round((time.time()-start_time)/60., 4), "minutes.")

## Using MICP and Gurobi

In [ ]:
start_time = time.time()
plan_through_buildings(save_location, runs, solve_gcs=False, solver=GurobiSolver(), file_addendum="gurobi")
print("Solved", runs, "buildings in", np.round((time.time()-start_time)/60., 4), "minutes.")

# Run statistical analysis

In [ ]:
failed_solves = []

relax_costs = np.empty(runs - len(failed_solves))
rounded_costs = np.empty(runs - len(failed_solves))
mip_costs = np.empty(runs - len(failed_solves))
relaxation_solver_time = np.empty(runs - len(failed_solves))
relaxation_time = np.empty(runs - len(failed_solves))
mip_solver_time = np.empty(runs - len(failed_solves))
mip_time = np.empty(runs - len(failed_solves))

ii = 0
for index in range(runs):
    if index in failed_solves:
        continue
    save_location = os.path.join(GcsDir(), "data/uav/statistics/room_" + str(index).zfill(3))
    with open(save_location + '/relaxation_plan_results.pkl', "rb") as f:
        data = pickle.load(f)
        relax_costs[ii] = data["relaxation_cost"]
        rounded_costs[ii] = data["rounded_cost"]
        relaxation_solver_time[ii] = data["gcs_solver_time"]
        relaxation_time[ii] = data["gcs_time"]
        
    with open(save_location + '/mip_plan_results.pkl', "rb") as f:
        data = pickle.load(f)
        mip_costs[ii] = data["mip_cost"]
        mip_solver_time[ii] = data["mip_total_solver_time"]
        mip_time[ii] = data["mip_time"]
    with open(save_location + '/mip_plan_results_gurobi.pkl', "rb") as f:
        data = pickle.load(f)
        if not np.isnan(data["mip_cost"]):
            if mip_costs[ii] > data["mip_cost"]:
                mip_costs[ii] = data["mip_cost"]
                mip_solver_time[ii] = data["mip_total_solver_time"]
                mip_time[ii] = data["mip_time"]
        
    ii += 1
    
mip_costs = np.minimum(mip_costs, rounded_costs)
    
rounding_gap = (rounded_costs-relax_costs)/relax_costs
relaxation_gap = (mip_costs - relax_costs)/mip_costs
solution_gap = (rounded_costs-mip_costs)/mip_costs

optimality_tolerance = 0.01

print("Tight Rounding:", np.sum(rounding_gap < optimality_tolerance)/ii)
print("Mean rounding gap:", np.mean(rounding_gap))
print("Max rounding gap:", np.argmax(rounding_gap), np.max(rounding_gap))
print()

print("Tight relaxation:", np.sum(relaxation_gap < optimality_tolerance)/ii)
print("Mean relaxation gap:", np.mean(relaxation_gap))
print("Max relaxation gap:", np.argmax(relaxation_gap), np.max(relaxation_gap))
print()

print("Solved to optimality:", np.sum(solution_gap < optimality_tolerance)/ii)
print("Mean solution gap:", np.mean(solution_gap))
print("Max solution gap:", np.argmax(solution_gap), np.max(solution_gap))

print()
print("Mean Rounding Solver Time:", np.mean(relaxation_solver_time))
print("Mean MIP Solver Time:", np.mean(mip_solver_time))
print("Mean MIP/Rounding Solver Time:", np.mean(mip_solver_time/relaxation_solver_time))


# Visualize a Trajectory from the Dataset

In [ ]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()

meshcat.SetProperty("/Grid", "visible", False)
meshcat.SetProperty("/Axes", "visible", False)
meshcat.SetProperty("/Lights/AmbientLight/<object>", "intensity", 0.8)
meshcat.SetProperty("/Lights/PointLightNegativeX/<object>", "intensity", 0)
meshcat.SetProperty("/Lights/PointLightPositiveX/<object>", "intensity", 0)

In [ ]:
view_relaxation = True
view_regions = False
view_traces = False
track_uav = False
save_html = False
room = 0

save_location = os.path.join(GcsDir(), "data/uav/statistics/room_" + str(room).zfill(3))
rounded_traj_file = save_location + "/relaxation_traj.pkl"
mip_traj_file = save_location + "/mip_traj.pkl"

# Load data from disk
shutil.copy(save_location + "/building.sdf", FindModelFile("models/room_gen/building.sdf"))

if view_regions:
    with open(save_location + "/regions.reg", "rb") as f:
        regions = pickle.load(f)

if view_relaxation:
    with open(rounded_traj_file, "rb") as f:
        b_traj = pickle.load(f)
else:
    with open(mip_traj_file, "rb") as f:
        b_traj = pickle.load(f)

# Build and run Diagram
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)

parser = Parser(plant, scene_graph)
parser.package_map().Add("gcs", GcsDir())
model_id = parser.AddModelFromFile(FindModelFile("models/room_gen/building.sdf"))

plant.Finalize()

meshcat_cpp = MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)

if b_traj is not None:
    animator = meshcat_cpp.StartRecording()
    if not track_uav:
        animator = None
    traj_system = builder.AddSystem(FlatnessInverter(b_traj, animator))
    quad = QuadrotorGeometry.AddToBuilder(builder, traj_system.get_output_port(0), scene_graph)
diagram = builder.Build()

# Set up a simulator to run this diagram
simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)

meshcat.Delete()

if view_traces:
    with open(rounded_traj_file, "rb") as f:
        rounded_traj = pickle.load(f)
    with open(mip_traj_file, "rb") as f:
        mip_traj = pickle.load(f)
        
    samples = 15000
    rounded_knots = rounded_traj.vector_values(np.linspace(rounded_traj.start_time(), rounded_traj.end_time(), samples))
    mip_knots = mip_traj.vector_values(np.linspace(mip_traj.start_time(), mip_traj.end_time(), samples))
    
    radius = 0.1
    rounded_pointcloud = PointCloud(samples)
    mip_pointcloud = PointCloud(samples)
    rounded_pointcloud.mutable_xyzs()[:] = rounded_knots[:]
    mip_pointcloud.mutable_xyzs()[:] = mip_knots[:]
    meshcat.SetObject("trace/rounded", rounded_pointcloud, radius, rgba=Rgba(0.0, 0.0, 1.0, 1.0))
    meshcat.SetObject("trace/mip", mip_pointcloud, radius, rgba=Rgba(1.0, 0.749, 0.0, 1.0))

if view_regions:
    for ii in range(len(regions)):
        v = VPolytope(regions[ii])
        meshcat.SetTriangleMesh("iris/region_" + str(ii), v.vertices(),
                                ConvexHull(v.vertices().T).simplices.T, Rgba(0.698, 0.67, 1, 0.4))
        
# Simulate
if b_traj is not None:
    end_time = b_traj.end_time()
    simulator.AdvanceTo(end_time+0.05)
    meshcat_cpp.PublishRecording()
    
    if save_html:
        with open (os.path.join(GcsDir(), "data/room_gen/uav_trajectory.html"), "w") as f:
           f.write(meshcat.StaticHtml())
else:
    simulator.AdvanceTo(0.1)